<a href="https://colab.research.google.com/github/tanvircr7/meh/blob/master/01_CODE_pytorch_dbourke.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>